# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. 

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# load the salaries dataset

salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [2]:
# examine the first 5 rows of the dataset
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"ADRIANO, RACQUEL ANNE",PARAMEDIC,FIRE,F,Salary,NaN,72510.0,NaN
1,"AFFANEH, MAHIR A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
2,"AFOENYI, PHYLLIS",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,13.0
3,"AGAR, BULENT B",DEPUTY COMMISSIONER,WATER MGMNT,F,Salary,NaN,132972.0,NaN
4,"AGEE, LAWRENCE",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [3]:
# count the number of missing values in each column
print(salaries.isnull().sum())

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25528
Annual Salary         8174
Hourly Rate          25528
dtype: int64


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [4]:
# count the number of salaried and hourly employees
print("Number of salaried employees: ", salaries[salaries['Salary or Hourly'] == 'Salary'].count()[0])
print("Number of hourly employees: ", salaries[salaries['Salary or Hourly'] == 'Hourly'].count()[0])

Number of salaried employees:  25528
Number of hourly employees:  8174


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [5]:
# list all departments and the count of employees in each department
print(salaries['Department'].value_counts())

POLICE                 13848
FIRE                    4627
STREETS & SAN           2175
WATER MGMNT             1914
AVIATION                1856
OEMC                    1737
TRANSPORTN              1113
PUBLIC LIBRARY          1088
DAIS                    1033
FAMILY & SUPPORT         656
FINANCE                  544
HEALTH                   467
LAW                      398
CITY COUNCIL             362
BUILDINGS                258
PUBLIC SAFETY ADMIN      244
BUSINESS AFFAIRS         167
HOUSING & ECON DEV       154
COPA                     123
BOARD OF ELECTION        111
MAYOR'S OFFICE           104
INSPECTOR GEN             95
CITY CLERK                88
PROCUREMENT               84
HUMAN RESOURCES           75
ANIMAL CONTRL             75
CULTURAL AFFAIRS          74
HOUSING                   66
BUDGET & MGMT             44
ADMIN HEARNG              38
DISABILITIES              28
TREASURER                 28
HUMAN RELATIONS           17
BOARD OF ETHICS            8
POLICE BOARD  

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [6]:
# import usefull libraries
from scipy import stats

# Select only the rows for hourly employees
hourly = salaries[salaries["Salary or Hourly"] == "Hourly"]

# Extract the Hourly Rate column
hourly_rates = hourly["Hourly Rate"]

# Perform the one-sample t-test to test the hypothesis that the mean hourly rate is equal to $30
t_statistic, p_value = stats.ttest_1samp(hourly_rates, 30)

# Print the p-value and test statistic
print("p-value:", p_value)
print("test statistic:", t_statistic)

# If the p-value is less than 0.05, we reject the null hypothesis that the mean hourly rate is equal to $30
if p_value < 0.05:
    print("Reject null hypothesis. Mean hourly rate is significantly different from $30.")
else:
    print("Fail to reject null hypothesis. Mean hourly rate is not significantly different from $30.")


p-value: 3.825475847673794e-202
test statistic: 31.211500329216044
Reject null hypothesis. Mean hourly rate is significantly different from $30.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [7]:
# One-sided test:
# H0: mean of police's salaries this year are <= $86000/year
# H1: mean of police's salaries this year are > $86000/year


# Filter the data to include only salaried employees in the police department
police_salaries = salaries[(salaries['Job Titles'] == 'POLICE OFFICER') & (salaries['Salary or Hourly'] == 'Salary')]

# Calculate the mean salary of the police department
police_mean = police_salaries['Annual Salary'].mean()

# Perform a one-tailed t-test with a 95% confidence interval
t_test_results = stats.ttest_1samp(police_salaries['Annual Salary'], 86000)
t_statistic = t_test_results[0]
p_value = t_test_results[1] / 2 # One-tailed p-value is half of two-tailed p-value

# Print the t-statistic and p-value
print("t-statistic:", t_statistic)
print("p-value:", p_value)

# Check if the p-value is less than 0.05, which means we reject the null hypothesis
if p_value < 0.05:
    print("Reject the null hypothesis - the mean salary of police officers is significantly different from $86000/year")
else:
    print("Fail to reject the null hypothesis - the mean salary of police officers is not significantly different from $86000/year")


t-statistic: -26.727780708452602
p-value: 3.011006959218849e-152
Reject the null hypothesis - the mean salary of police officers is significantly different from $86000/year


Using the crosstab function, find the department that has the most hourly workers.

In [8]:
# Create a cross tabulation of the departments and employee type (hourly or salaried)
department_employee_type = pd.crosstab(index=salaries['Department'], columns=salaries['Salary or Hourly'])

# Find the department with the most hourly workers
most_hourly_workers_department = department_employee_type['Hourly'].idxmax()

# Print the department with the most hourly workers
print(f"The department with the most hourly workers is {most_hourly_workers_department}.")


The department with the most hourly workers is STREETS & SAN.


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [9]:
# Select only the hourly workers
hourly_workers = salaries[salaries["Salary or Hourly"] == "Hourly"]

# Create a crosstab to find the department with the most hourly workers
most_hourly_workers = pd.crosstab(index=hourly_workers["Department"], 
                                  columns="count")
most_hourly_workers = most_hourly_workers.sort_values(by='count', ascending=False)

# Find the department with the most hourly workers
department_with_most_hourly_workers = most_hourly_workers.index[0]

# Select only the hourly workers from the department with the most hourly workers
department_hourly_workers = hourly_workers[hourly_workers["Department"] == department_with_most_hourly_workers]

# Perform the one sample t-test
t_statistic, p_value = stats.ttest_1samp(department_hourly_workers["Hourly Rate"], 35)

# Print the results
print("t-statistic: ", t_statistic)
print("p-value: ", p_value)

# Interpret the results
if p_value < 0.05:
    print("We reject the null hypothesis. There is evidence that the hourly wage of workers in the department with the most hourly workers is less than $35/hour.")
else:
    print("We fail to reject the null hypothesis. There is not enough evidence to suggest that the hourly wage of workers in the department with the most hourly workers is less than $35/hour.")


t-statistic:  4.000138803968834
p-value:  6.580474443773579e-05
We reject the null hypothesis. There is evidence that the hourly wage of workers in the department with the most hourly workers is less than $35/hour.


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [10]:
from scipy import stats

# Extract hourly wage data
hourly_wage = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate']

# Compute the mean and standard error of the mean
mean = np.mean(hourly_wage)
sem = stats.sem(hourly_wage)

# Compute the degrees of freedom
dof = len(hourly_wage) - 1

# Compute the confidence interval
confidence_interval = stats.t.interval(alpha = 0.95,              # Confidence level
                                      df = dof,                  # Degrees of freedom
                                      scale = sem,               # Standard error
                                      loc = mean)                # Mean

# Print the confidence interval
print("The 95% confidence interval of the mean hourly wage is", confidence_interval)


The 95% confidence interval of the mean hourly wage is (34.29839539204361, 34.874504045197305)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [11]:
import scipy.stats as stats

# Filter the data to only include police department salaried employees
police_salaried = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

# Compute the mean of the salaried police employees
mean_salary = police_salaried['Annual Salary'].mean()

# Compute the standard error of the mean of the salaried police employees
sem = stats.sem(police_salaried['Annual Salary'])

# Compute the degrees of freedom
dof = len(police_salaried) - 1

# Compute the confidence interval
conf_int = stats.t.interval(alpha=0.95, df=dof, loc=mean_salary, scale=sem)

print("95% Confidence interval for the mean annual salary of police salaried employees: ", conf_int)


95% Confidence interval for the mean annual salary of police salaried employees:  (88534.1182885883, 89134.11955863392)


##### CONFIDENCE INTERVAL IS (88534.1182885883, 89134.11955863392)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [12]:
from statsmodels.stats.proportion import proportions_ztest

proportions_ztest(salaries[salaries['Salary or Hourly'] == 'Salary']['Salary or Hourly'].count(),
                  len(salaries['Salary or Hourly']),0.75)


(3.196245387033488, 0.0013922864704323692)

In [13]:
(salaries[salaries['Salary or Hourly'] == 'Salary']['Salary or Hourly'].count())/(salaries['Salary or Hourly'].count())

0.7574624651356002